<a href="https://colab.research.google.com/github/Deng-Xian-Sheng/Real-technology/blob/main/%E2%80%9CDeepSeek_R1_Distill_Qwen_1_5B_Conversational_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

要在免费的 Tesla T4 Google Colab 实例上运行此程序，请点击“运行时”，然后点击“全部运行”。

   如果您需要帮助，请加入 Discord + ⭐ 在 Github 上给我们点赞 ⭐
要在您自己的电脑上安装 Unsloth，请按照我们 Github 页面上的安装说明进行操作 [redacted link]。

您将学习如何进行[redacted link]、如何[redacted link]、如何[redacted link] 以及[redacted link]。

访问我们的文档以获取我们所有的[redacted link] 和[redacted link]。

### News

[新增] 我们修复了 Phi-4 中的许多错误，这大大提高了 Phi-4 的准确性。请参阅我们的[redacted link]。

[新增]** 您可以查看所有包含我们错误修复的 Phi-4 模型上传，包括[redacted link]、GGUF 等[redacted link]。

[新增] 截至 2024 年 11 月，Unsloth 现在支持[redacted link]！

希望这可以帮到您！如果您有任何其他问题，请告诉我。

### Installation

In [ ]:
%%capture
# 通常情况下，使用 pip install unsloth 就足够了

# 截至 2025 年 1 月 31 日，Colab 暂时存在一些与 Pytorch 相关的问题
# 使用 pip install unsloth 将花费 3 分钟，而以下方法花费不到 1 分钟：
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

### Unsloth

In [ ]:
from unsloth import FastLanguageModel  # 从 unsloth 库导入 FastLanguageModel 类
import torch  # 导入 PyTorch 库

max_seq_length = 2048  # 设置最大序列长度，此处为 2048，支持 RoPE Scaling 自动缩放
dtype = None  # 数据类型，设置为 None 自动检测，Tesla T4、V100 使用 Float16，Ampere+ 使用 Bfloat16
load_in_4bit = True  # 使用 4 位量化减少内存使用，可以设置为 False

# 支持的 4 位预量化模型，下载速度提升 4 倍，避免内存不足错误
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",  # Llama-3.1，速度提升 2 倍
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Mistral-Small-Instruct-2409",  # Mistral 22b，速度提升 2 倍！
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",  # Phi-3.5，速度提升 2 倍！
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-27b-bnb-4bit",  # Gemma，速度提升 2 倍！
    "unsloth/Llama-3.2-1B-bnb-4bit",  # 全新！Llama 3.2 模型
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
]  # 更多模型请访问 https://huggingface.co/unsloth

qwen_models = [
    "unsloth/Qwen2.5-Coder-32B-Instruct",  # Qwen 2.5 Coder，速度提升 2 倍
    "unsloth/Qwen2.5-Coder-7B",
    "unsloth/Qwen2.5-14B-Instruct",  # 14B 参数模型适用于 16GB 显卡
    "unsloth/Qwen2.5-7B",
    "unsloth/Qwen2.5-72B-Instruct",  # 72B 参数模型适用于 48GB 显卡
    "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit"
]  # 更多模型请访问 https://huggingface.co/unsloth

# 从预训练模型加载模型和分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit",  # 模型名称
    max_seq_length=max_seq_length,  # 最大序列长度
    dtype=dtype,  # 数据类型
    load_in_4bit=load_in_4bit,  # 是否使用 4 位量化加载
    # token="hf_...",  # 如果使用 gated 模型（如 meta-llama/Llama-2-7b-hf），请使用此参数
)

我们现在添加 LoRA 适配器，这样我们只需要更新 1% 到 10% 的所有参数！

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # 选择任意大于 0 的数字！建议值：8、16、32、64、128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],  # 应用 LoRA 的目标模块
    lora_alpha = 16,  # LoRA 的缩放因子
    lora_dropout = 0,  # 支持任意值，但 0 是最优化的
    bias = "none",  # 支持任意值，但 "none" 是最优化的
    # [新功能] "unsloth" 减少 30% 的显存使用，适用于 2 倍大的批次大小！
    use_gradient_checkpointing = "unsloth",  # 对于非常长的上下文，设置为 True 或 "unsloth"
    random_state = 3407,  # 随机种子
    use_rslora = False,  # 支持秩稳定 LoRA
    loftq_config = None,  # 支持 LoftQ
)

## 数据准备
现在，我们使用 Qwen-2.5 格式进行对话风格的微调。我们使用 mlabonne/FineTome-100k 数据集，该数据集采用 ShareGPT 风格。但是，我们将其转换为 HuggingFace 的标准多轮对话格式 ("role", "content")，而不是 ("from", "value")。Qwen 渲染多轮对话的方式如下：

```
<|im_start|>system
你是文心一言，由阿里云创造。你是一个乐于助人的助手。<|im_end|>
<|im_start|>user
2+2 等于多少？<|im_end|>
<|im_start|>assistant
等于 4。<|im_end|>
```
我们使用 get_chat_template 函数来获取正确的对话模板。我们支持 zephyr、chatml、mistral、llama、alpaca、vicuna、vicuna_old、phi3、llama3 等多种对话模板。

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)


In [ ]:
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

In [ ]:
# 挂载谷歌drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
"""
######
#从json加载数据集
######

如果你的数据集是：
[
  [
    {
      "role":"",
      "content":""
    }
  ]
]

则需要：
import json
from datasets import Dataset
# 手动加载 JSON 文件，将每个对话列表包装到 "conversations" 字段
with open('./drive/MyDrive/Colab Notebooks/xiaoshuo_dataset.json', 'r') as f:
  data = json.load(f)
  # 将每个对话列表包装为 {"conversations": [...]}
  wrapped_data = [{"conversations": convo} for convo in data]
  dataset = Dataset.from_list(wrapped_data)

请尽可能的预先把数据集做成：
[
  {
    "conversations": [
      {
        "role":"",
        "content":"<think>嗯，这个问题……</think>你需要……"
      }
    ]
  }
]
"""

"""
######
#从HF加载数据集
######
from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")
"""

# 从json加载数据集但符合格式
from datasets import load_dataset
dataset = load_dataset("json",data_files='./drive/MyDrive/Colab Notebooks/xiaoshuo_dataset.json', split = "train")

我们现在使用 standardize_sharegpt 将 ShareGPT 风格的数据集转换为 HuggingFace 的通用格式。这会将数据集从以下形式：

```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
转换为以下形式：
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```


In [ ]:
from unsloth.chat_templates import standardize_sharegpt
# dataset = standardize_sharegpt(dataset) # 禁用此行，除非你需要转换
dataset = dataset.map(formatting_prompts_func, batched = True,)

我们来看看项目5的对话结构：

In [ ]:
dataset[5]["conversations"]

然后我们看看聊天模板是如何转换这些对话的。

In [ ]:
dataset[5]["text"]

# 模型训练
现在，让我们使用 Huggingface TRL 库的 SFTTrainer！更多文档请参考：[redacted link]。

为了加快训练速度，我们只进行 60 步迭代，但您可以设置 num_train_epochs=1 进行完整训练，并注释max_steps或设为0。

我们也支持 TRL 的 DPOTrainer！

此训练器包含我们对 梯度累积错误 的修复。想了解更多信息，请阅读：[redacted link]

In [ ]:
from trl import SFTTrainer  # 从 trl 库导入 SFTTrainer 类，用于监督式微调
from transformers import TrainingArguments, DataCollatorForSeq2Seq  # 从 transformers 库导入训练参数和数据整理器
from unsloth import is_bfloat16_supported  # 从 unsloth 库导入 is_bfloat16_supported 函数，用于检查是否支持 bf16 数据类型


trainer = SFTTrainer(  # 创建一个 SFTTrainer 实例
    model=model,  # 要微调的模型
    tokenizer=tokenizer,  # 分词器
    train_dataset=dataset,  # 训练数据集
    dataset_text_field="text",  # 数据集中文本字段的名称
    max_seq_length=max_seq_length,  # 最大序列长度
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),  # 数据整理器，用于将数据整理成模型所需的格式
    dataset_num_proc=4,  # 用于数据处理的进程数
    packing=False,  # 是否打包数据，对于短序列可以加快训练速度 5 倍
    args=TrainingArguments(  # 训练参数
        per_device_train_batch_size=1,  # 每个设备的训练批次大小
        # per_device_train_batch_size=4,  # 每个设备的训练批次大小
        gradient_accumulation_steps=4,  # 梯度累积步数，最新版 Unsloth 修复了一个主要错误
        # num_train_epochs=1,  # 训练轮数，设置为 1 进行完整训练
        warmup_steps=5,  # 预热步数
        max_steps=30,  # 最大训练步数
        # warmup_steps=666,  # 预热步数
        # max_steps=4000,  # 最大训练步数
        learning_rate=2e-4,  # 学习率
        fp16=not is_bfloat16_supported(),  # 是否使用 fp16 数据类型
        bf16=is_bfloat16_supported(),  # 是否使用 bf16 数据类型
        logging_steps=1,  # 日志记录步数
        optim="paged_adamw_8bit",  # 优化器，使用 paged_adamw_8bit 节省更多内存
        weight_decay=0.01,  # 权重衰减
        lr_scheduler_type="linear",  # 学习率调度器类型
        seed=3407,  # 随机种子
        output_dir="outputs",  # 输出目录
        report_to="none",  # 是否将训练报告发送到其他平台，例如 WandB
        # report_to="WandB",  # 是否将训练报告发送到其他平台，例如 WandB
    ),
)

我们还使用 Unsloth 的 train_on_completions 方法，仅对助手的输出进行训练，而忽略用户输入的损失。

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

我们验证掩码是否已实际应用：

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

我们可以看到系统和指令提示已被成功屏蔽！

In [ ]:
# @title 显示当前内存统计信息
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

我们修复了所有训练器中的一个主要的梯度累积错误。有关更多详细信息，请参阅[redacted link]。

In [ ]:
# 开始训练
trainer_stats = trainer.train()

In [ ]:
# @title 显示最终内存和时间统计信息
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

## 推理
让我们运行模型！您可以更改指令和输入 - 将输出留空！

[新功能] 在免费的 Colab 中尝试使用 Llama-3.1 8b Instruct 进行 2 倍速推理 [此处](已删除链接)

我们使用 min_p = 0.1 和 temperature = 1.5。阅读此 [推文] (已删除链接)以了解更多信息。

In [ ]:
from unsloth.chat_templates import get_chat_template  # 从 unsloth 库导入 get_chat_template 函数

# 获取 Qwen-2.5 的聊天模板，并应用到分词器
tokenizer = get_chat_template(
    tokenizer,
    chat_template="qwen-2.5",
)

FastLanguageModel.for_inference(model)  # 启用原生 2 倍速推理

# 定义消息列表，包含用户角色和内容
messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},  # 用户要求继续斐波那契数列
]

# 使用聊天模板处理消息，进行分词，添加生成提示，并转换为 PyTorch 张量
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,  # 必须添加用于生成
    return_tensors="pt",
).to("cuda")  # 将张量移动到 CUDA 设备（GPU）

# 使用模型生成输出
outputs = model.generate(
    input_ids=inputs,  # 输入 ID
    max_new_tokens=64,  # 最大生成的新标记数
    use_cache=True,  # 使用缓存加速生成
    temperature=1.5,  # 控制生成结果的随机性
    min_p=0.1,  # 控制生成结果的多样性
)

# 将模型输出解码为文本
tokenizer.batch_decode(outputs)

您也可以使用 TextStreamer 进行连续推理 - 这样您就可以逐个标记地看到生成结果，而不是一直等待！

In [ ]:
FastLanguageModel.for_inference(model) # 启用原生 2 倍速推理

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},  # 用户消息：继续斐波那契数列
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,  # 对消息进行分词
    add_generation_prompt = True,  # 必须添加用于生成
    return_tensors = "pt",  # 返回 PyTorch 张量
).to("cuda")  # 将张量移动到 CUDA 设备（GPU）

from transformers import TextStreamer  # 导入 TextStreamer 用于连续推理
text_streamer = TextStreamer(tokenizer, skip_prompt = True)  # 创建 TextStreamer 实例
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,  # 使用模型生成输出
                   use_cache = True, temperature = 1.5, min_p = 0.1)  # 设置生成参数

### 保存和加载微调模型
要将最终模型保存为 LoRA 适配器，可以使用 Huggingface 的 push_to_hub 进行在线保存，或使用 save_pretrained 进行本地保存。

[注意] 这只会保存 LoRA 适配器，而不是完整的模型。要保存为 16 位或 GGUF 格式，请向下滚动！

In [ ]:
model.save_pretrained("lora_model")  # 本地保存
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # 在线保存
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # 在线保存

现在，如果您想加载我们刚刚保存的 LoRA 适配器进行推理，请将 False 设置为 True：

In [ ]:
if False:  # 这段代码目前不会被执行，因为条件为 False
    from unsloth import FastLanguageModel  # 从 unsloth 库导入 FastLanguageModel 类
    # 从预训练模型加载模型和分词器
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="lora_model",  # 你用于训练的模型名称，应替换为实际的模型名称
        max_seq_length=max_seq_length,  # 最大序列长度，之前已定义
        dtype=dtype,  # 数据类型，之前已定义
        load_in_4bit=load_in_4bit,  # 是否使用 4 位量化加载，之前已定义
    )
    FastLanguageModel.for_inference(model)  # 启用原生 2 倍速推理

# 定义消息列表，包含用户角色和内容
messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},  # 用户消息：描述法国首都的一座高塔
]

# 使用聊天模板处理消息，进行分词，添加生成提示，并转换为 PyTorch 张量
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,  # 对消息进行分词
    add_generation_prompt=True,  # 必须添加用于生成
    return_tensors="pt",  # 返回 PyTorch 张量
).to("cuda")  # 将张量移动到 CUDA 设备（GPU）

from transformers import TextStreamer  # 导入 TextStreamer 用于连续推理
text_streamer = TextStreamer(tokenizer, skip_prompt=True)  # 创建 TextStreamer 实例
# 使用模型生成输出
_ = model.generate(
    input_ids=inputs,  # 输入 ID
    streamer=text_streamer,  # 使用 TextStreamer 进行连续推理
    max_new_tokens=128,  # 最大生成的新标记数
    use_cache=True,  # 使用缓存加速生成
    temperature=1.5,  # 控制生成结果的随机性
    min_p=0.1,  # 控制生成结果的多样性
)

您也可以使用 Hugging Face 的 AutoModelForPeftCausalLM。仅当您未安装 unsloth 时才使用此方法。它可能会非常慢，因为它不支持 4bit 模型下载，而且 Unsloth 的 推理速度快 2 倍。



In [ ]:
if False:  # 这段代码目前不会被执行，因为条件为 False
    # 我强烈不建议这样做 - 如果可能，请使用 Unsloth
    from peft import AutoPeftModelForCausalLM  # 从 peft 库导入 AutoPeftModelForCausalLM 类
    from transformers import AutoTokenizer  # 从 transformers 库导入 AutoTokenizer 类

    # 从预训练模型加载模型，使用 LoRA 适配器
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model",  # 你用于训练的模型名称，应替换为实际的模型名称
        load_in_4bit=load_in_4bit,  # 是否使用 4 位量化加载，之前已定义
    )
    # 从预训练模型加载分词器
    tokenizer = AutoTokenizer.from_pretrained("lora_model")  # 你用于训练的模型名称，应替换为实际的模型

## 为 VLLM 保存为 float16 格式
我们也支持直接保存为 float16 格式。选择 merged_16bit 用于 float16 或 merged_4bit 用于 int4。我们也允许使用 lora 适配器作为备用方案。使用 push_to_hub_merged 上传到您的 Hugging Face 帐户！您可以访问 [redacted link] 获取您的个人令牌。

In [ ]:
# 合并到 16 位
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# 合并到 4 位
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# 仅 LoRA 适配器
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

## GGUF / llama.cpp 转换
为了保存到 GGUF / llama.cpp，我们现在原生支持它！我们克隆了 llama.cpp，默认保存到 q8_0。我们允许所有方法，例如 q4_k_m。使用 save_pretrained_gguf 进行本地保存，使用 push_to_hub_gguf 上传到 HF。

一些支持的量化方法（完整列表请参见我们的 [redacted link]）：

 - q8_0 - 快速转换。资源占用高，但通常可以接受。
 - q4_k_m - 推荐。对 attention.wv 和 feed_forward.w2 张量的一半使用 Q6_K，否则使用 Q4_K。
 - q5_k_m - 推荐。对 attention.wv 和 feed_forward.w2 张量的一半使用 Q6_K，否则使用 Q5_K。

[新增] 要微调并自动导出到 Ollama，请尝试我们的 [redacted link]

In [ ]:
# 保存为 8 位 Q8_0 格式
if False: model.save_pretrained_gguf("model", tokenizer,)
# 请记住访问 https://huggingface.co/settings/tokens 获取令牌！
# 并将 hf 更改为您的用户名！
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# 保存为 16 位 GGUF 格式
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# 保存为 q4_k_m GGUF 格式
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# 保存为多种 GGUF 格式 - 如果您需要多种格式，这样会更快！
if False:
    model.push_to_hub_gguf(
        "hf/model", # 将 hf 更改为您的用户名！
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # 在 https://huggingface.co/settings/tokens 获取令牌
    )

现在，在 llama.cpp 或基于 UI 的系统（如 Jan 或 Open WebUI）中使用 model-unsloth.gguf 文件或 model-unsloth-Q4_K_M.gguf 文件。您可以[redacted link]安装 Jan，[redacted link]安装 Open WebUI。
大功告成！如果您对 Unsloth 有任何疑问，我们有一个 [redacted link] 频道！如果您发现任何错误或想了解最新的 LLM 信息，或者需要帮助、加入项目等，请随时加入我们的 Discord！

其他一些链接：

Llama 3.2 对话笔记本。[redacted link]
将微调结果保存到 Ollama。[redacted link]
Llama 3.2 视觉微调 - 放射学用例。[redacted link]
在我们的[redacted link]中查看有关 DPO、ORPO、持续预训练、对话微调等内容的笔记本！
  
如果您需要帮助，请加入 Discord + ⭐️ 在 Github 上给我们点赞 ⭐️